In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import shutil



In [3]:
def extract_hoppings_from_hr(hr_path, dim=2):
    """
    Extract hopping terms from wannier90_hr.dat

    Parameters
    ----------
    hr_path : str or Path
        Path to wannier90_hr.dat
    dim : int
        Dimensionality of the model (2 or 3).
        For 2D, Rz will be ignored.

    Returns
    -------
    num_wann : int
        Number of Wannier orbitals
    onsite : np.ndarray (complex)
        On-site terms H_ii(R=0)
    hoppings : list of tuples
        Each element is (R, i, j, t),
        where:
            R : list[int]   (length = dim)
            i,j : int       (0-based)
            t : complex
    """
    hr_path = Path(hr_path)
    lines = hr_path.read_text().splitlines()

    # --- header ---
    num_wann = int(lines[1].strip())
    nrpts = int(lines[2].strip())

    # --- skip degeneracy lines ---
    idx = 3
    degeneracies = []
    while len(degeneracies) < nrpts:
        parts = lines[idx].split()
        for p in parts:
            if len(degeneracies) < nrpts:
                degeneracies.append(int(p))
        idx += 1

    onsite = np.zeros(num_wann, dtype=complex)
    hoppings = []

    # --- hopping data ---
    for line in lines[idx:]:
        parts = line.split()
        if len(parts) != 7:
            continue

        Rx, Ry, Rz = map(int, parts[:3])
        i, j = map(int, parts[3:5])
        re, im = map(float, parts[5:7])

        t = re + 1j * im
        i0, j0 = i - 1, j - 1  # Wannier90 is 1-based

        # on-site term
        if Rx == 0 and Ry == 0 and Rz == 0 and i0 == j0:
            onsite[i0] = t
        else:
            R = [Rx, Ry] if dim == 2 else [Rx, Ry, Rz]
            hoppings.append((R, i0, j0, t))

    return num_wann, onsite, hoppings


In [4]:
# ===== 晶格（2D 四方，单位 Å）=====
a0 = 4.1630263607523368

lat_vecs = [
    [a0, 0.0],
    [0.0, a0],
]

# ===== 轨道中心（分数坐标）=====
# 2 个磁性原子 × dz2 × (up, dn)
orb_vecs = [
    [0.5, 0.0],  # atom A, dz2 ↑
    [0.0, 0.5],  # atom B, dz2 ↑
    [0.5, 0.0],  # atom A, dz2 ↓
    [0.0, 0.5],  # atom B, dz2 ↓
]


In [5]:
workdir = Path(".").resolve()

dat_files = sorted(workdir.glob("wannier*.dat"))

if not dat_files:
    raise RuntimeError("No wannier*.dat files found in current directory.")

print(f"Found {len(dat_files)} hr.dat file(s):")
for f in dat_files:
    print("  -", f.name)


Found 559 hr.dat file(s):
  - wannier90_ie35_it1_ir23.dat
  - wannier90_ie35_it1_ir24.dat
  - wannier90_ie35_it1_ir25.dat
  - wannier90_ie35_it1_ir26.dat
  - wannier90_ie35_it1_ir27.dat
  - wannier90_ie35_it1_ir28.dat
  - wannier90_ie35_it1_ir29.dat
  - wannier90_ie35_it1_ir3.dat
  - wannier90_ie35_it1_ir30.dat
  - wannier90_ie35_it1_ir31.dat
  - wannier90_ie35_it1_ir32.dat
  - wannier90_ie35_it1_ir33.dat
  - wannier90_ie35_it1_ir34.dat
  - wannier90_ie35_it1_ir35.dat
  - wannier90_ie35_it1_ir36.dat
  - wannier90_ie35_it1_ir37.dat
  - wannier90_ie35_it1_ir38.dat
  - wannier90_ie35_it1_ir39.dat
  - wannier90_ie35_it1_ir4.dat
  - wannier90_ie35_it1_ir40.dat
  - wannier90_ie35_it1_ir41.dat
  - wannier90_ie35_it1_ir5.dat
  - wannier90_ie35_it1_ir6.dat
  - wannier90_ie35_it1_ir7.dat
  - wannier90_ie35_it1_ir8.dat
  - wannier90_ie35_it1_ir9.dat
  - wannier90_ie36_it1_ir1.dat
  - wannier90_ie36_it1_ir10.dat
  - wannier90_ie36_it1_ir11.dat
  - wannier90_ie36_it1_ir12.dat
  - wannier90_ie36_it1

In [ ]:
from pythtb import tb_model
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import shutil

# ==================================================
# 工作目录与输入文件
# ==================================================
workdir = Path(r"E:\马睿骁\组会汇报\Nb2OSSe\pythTB\workflow")
dat_files = sorted(workdir.glob("wannier90_*.dat"))

# ==================================================
# 记录非平庸陈数的文件（新增功能）
# ==================================================
chern_record_file = workdir / "nontrivial_chern_list.txt"
with open(chern_record_file, "w", encoding="utf-8") as f:
    f.write("# hr.dat with non-zero, well-defined Chern number\n")
    f.write("# format: filename    chern\n")

# ==================================================
# 相图标准 - 保留
# ==================================================
def classify_phase(evals, gap_min=1e-3):
    E = np.asarray(evals, float).ravel()
    Es = np.unique(np.sort(E))

    if Es.size < 10:
        return {"has_gap": False, "gap": 0.0, "midgap": None, "phase": "metal"}

    gaps = Es[1:] - Es[:-1]
    igap = int(np.argmax(gaps))
    gap = gaps[igap]

    if gap < gap_min:
        return {"has_gap": False, "gap": float(gap), "midgap": None, "phase": "metal"}

    VBM = Es[igap]
    CBM = Es[igap + 1]

    return {
        "has_gap": True,
        "gap": float(gap),
        "midgap": 0.5 * (VBM + CBM),
        "phase": "insulator",
    }

# ==================================================
# Hermitian hopping 的规范 R 判据 - 保留
# ==================================================
def canonical_R(R):
    R = tuple(R)
    return R > tuple(-r for r in R)

# ==================================================
# 高对称路径（仅用于相分类）- 保留
# ==================================================
path = [
    [0.0, 0.0],
    [0.5, 0.0],
    [0.5, 0.5],
    [0.0, 0.5],
    [0.0, 0.0],
]
nk = 301

# ==================================================
# 主循环
# ==================================================
for dat in dat_files:
    name = dat.stem
    calc_dir = workdir / name
    calc_dir.mkdir(exist_ok=True)

    print(f"\nProcessing {dat.name}")

    # --------------------------------------------------
    # 1) 读取 Wannier hr
    # --------------------------------------------------
    num_wann, onsite, hoppings = extract_hoppings_from_hr(dat, dim=2)

    if num_wann != len(orb_vecs):
        print(f"  [SKIP] Orbital mismatch: {num_wann} vs {len(orb_vecs)}")
        continue

    # --------------------------------------------------
    # 2) 构造 TB 模型
    # --------------------------------------------------
    model = tb_model(
        dim_k=2,
        dim_r=2,
        lat=lat_vecs,
        orb=orb_vecs,
    )

    model.set_onsite(onsite.real.tolist())

    nhop_used = 0
    for R, i, j, t in hoppings:
        R = tuple(R)
        if (i > j) or (i == j and not canonical_R(R)):
            continue
        model.set_hop(t, i, j, list(R))
        nhop_used += 1

    print(f"  [INFO] Used hoppings: {nhop_used}")

    # --------------------------------------------------
    # 3) Berry phase + 陈数（核心部分）
    # --------------------------------------------------
    try:
        from pythtb import WFArray, Mesh

        # === 2D 模型 → 2D mesh（关键一致性）===
        Nk_chern = 31
        mesh = Mesh(["k", "k"])
        mesh.build_grid((Nk_chern, Nk_chern), gamma_centered=True)

        wfa = WFArray(model.lattice, mesh)
        wfa.solve_model(model)

        # ---- Berry phase ----
        phi_0 = wfa.berry_phase(axis_idx=0, state_idx=[0], contin=True)
        phi_1 = wfa.berry_phase(axis_idx=0, state_idx=[1], contin=True)
        phi_occ = wfa.berry_phase(axis_idx=0, state_idx=range(2), contin=True)

        fig, ax = plt.subplots(figsize=(8, 6))
        ky = np.linspace(0, 1, len(phi_0))
        ax.plot(ky, phi_0, "ro-", label="Band 0", markersize=3)
        ax.plot(ky, phi_1, "go-", label="Band 1", markersize=3)
        ax.plot(ky, phi_occ, "bo-", label="Occupied (0,1)", markersize=3)

        ax.legend()
        ax.set_xlabel(r"$k_y$")
        ax.set_ylabel(r"Berry phase along $k_x$")
        ax.set_xlim(0.0, 1.0)
        ax.set_ylim(-7.0, 7.0)
        ax.yaxis.set_ticks([-2 * np.pi, -np.pi, 0, np.pi, 2 * np.pi])
        ax.set_yticklabels((r"$-2\pi$", r"$-\pi$", r"$0$", r"$\pi$", r"$2\pi$"))
        ax.set_title(f"Berry Phases - {name}")
        plt.tight_layout()

        fig.savefig(calc_dir / "berry_phases.png", dpi=300)
        fig.savefig(calc_dir / "berry_phases.pdf")
        plt.close(fig)

        # ---- 陈数 ----
        ch_band0 = wfa.chern_number(state_idx=[0], plane=(0, 1))
        ch_band1 = wfa.chern_number(state_idx=[1], plane=(0, 1))
        ch_occ   = wfa.chern_number(state_idx=range(2), plane=(0, 1))

        print(f"  [INFO] Chern band0 = {ch_band0}")
        print(f"  [INFO] Chern band1 = {ch_band1}")
        print(f"  [INFO] Chern occupied (0,1) = {ch_occ}")

        # 兼容原有 I/O
        chern0 = ch_occ
        chern1 = ch_occ

        # --------------------------------------------------
        # ⭐ 新增功能：记录非平庸陈数（≠0 且 非 NaN）
        # --------------------------------------------------
        if isinstance(chern0, (int, float)):
            if not np.isnan(chern0) and abs(chern0) > 1e-6:
                with open(chern_record_file, "a", encoding="utf-8") as f:
                    f.write(f"{dat.name}\tchern={chern0:.6f}\n")

                print(f"  [TOPO] Non-trivial Chern detected: {dat.name}, C = {chern0}")

    except Exception as e:
        print(f"  [ERROR] Failed to compute topological invariants: {e}")
        chern0 = "NaN"
        chern1 = "NaN"

    # --------------------------------------------------
    # 4) 能带（仅用于相分类）
    # --------------------------------------------------
    k_vec, k_dist, k_node = model.k_path(path, nk, report=False)
    evals = model.solve_ham(k_pts=k_vec).T

    phase_info = classify_phase(evals)

    info_file = calc_dir / "phase_info.txt"
    with open(info_file, "a", encoding="utf-8") as f:
        f.write("\n# ==========================================\n")
        f.write("# Topological Invariants\n")
        f.write("# ==========================================\n")
        f.write(f"chern_occ = {chern0}\n")
        f.write("# ==========================================\n")

    # --------------------------------------------------
    # 5) 移动 hr 文件
    # --------------------------------------------------
    hr_target = calc_dir / dat.name
    if not hr_target.exists():
        shutil.move(str(dat), hr_target)

    print(f"  [OK] Finished {calc_dir.name}/")


In [9]:
from pathlib import Path
import re

# ==================================================
# 工作目录：当前工作目录（适用于 Jupyter/IPython）
# ==================================================
base_dir = Path.cwd()

out_file = base_dir / "chern_occ_summary.txt"

chern_pattern = re.compile(
    r"chern_occ\s*=\s*([+-]?\d+(\.\d+)?([eE][+-]?\d+)?)"
)

results = []

for d in sorted(base_dir.iterdir()):
    if not d.is_dir():
        continue
    if not d.name.startswith("wannier"):
        continue

    phase_file = d / "phase_info.txt"
    if not phase_file.exists():
        print(f"[SKIP] {d.name}: no phase_info.txt")
        continue

    text = phase_file.read_text(encoding="utf-8", errors="ignore")
    match = chern_pattern.search(text)
    if match is None:
        print(f"[WARN] {d.name}: chern_occ not found")
        continue

    chern_val = float(match.group(1))
    results.append((d.name, chern_val))

with open(out_file, "w", encoding="utf-8") as f:
    f.write("# folder_name\tchern_occ\n")
    for name, ch in results:
        f.write(f"{name}\t{ch:.12g}\n")

print("\n===== chern_occ summary =====")
for name, ch in results:
    print(f"{name:40s}  chern_occ = {ch:.6g}")

print(f"\n[OK] Results written to: {out_file}")


[WARN] wannier90_ie34_it1_ir41: chern_occ not found
[WARN] wannier90_ie34_it1_ir5: chern_occ not found
[WARN] wannier90_ie34_it1_ir6: chern_occ not found
[WARN] wannier90_ie34_it1_ir7: chern_occ not found
[WARN] wannier90_ie34_it1_ir8: chern_occ not found
[WARN] wannier90_ie34_it1_ir9: chern_occ not found
[WARN] wannier90_ie35_it1_ir1: chern_occ not found
[WARN] wannier90_ie35_it1_ir10: chern_occ not found
[WARN] wannier90_ie35_it1_ir11: chern_occ not found
[WARN] wannier90_ie35_it1_ir12: chern_occ not found
[WARN] wannier90_ie35_it1_ir13: chern_occ not found
[WARN] wannier90_ie35_it1_ir14: chern_occ not found
[WARN] wannier90_ie35_it1_ir15: chern_occ not found
[WARN] wannier90_ie35_it1_ir16: chern_occ not found
[WARN] wannier90_ie35_it1_ir17: chern_occ not found
[WARN] wannier90_ie35_it1_ir18: chern_occ not found
[WARN] wannier90_ie35_it1_ir19: chern_occ not found
[WARN] wannier90_ie35_it1_ir2: chern_occ not found
[WARN] wannier90_ie35_it1_ir20: chern_occ not found

===== chern_occ su

In [7]:
import pythtb
print(pythtb.__version__)
print(dir(pythtb))


2.0.0
['Lattice', 'Mesh', 'TBModel', 'W90', 'WFArray', 'Wannier', '_DEFAULT_HANDLER', '_DEFAULT_LOG_FORMAT', '_LOGGER_NAME', '__all__', '__annotations__', '__author__', '__builtins__', '__cached__', '__doc__', '__file__', '__license__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_coerce_level', 'configure_logging', 'finite_diff_coeffs', 'finite_difference', 'get_trial_wfs', 'hoptable', 'io', 'is_Hermitian', 'lattice', 'levi_civita', 'logger', 'logging', 'm', 'mesh', 'pauli_decompose', 'set_log_level', 'tb_model', 'tbmodel', 'utils', 'visualization', 'w90', 'wannier', 'wfarray']
